In [2]:
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('../bbc_small.csv', sep=';', index_col=0)
df

,index,title,subtitle,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_min,topic,image_l,image_xl,cleaned_syno_l,k_means
0,0,BBC Proms,"2021: Last Night of the Proms, Part 2",Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,BBC RADIO THREE,False,2021,87,music,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,katie derham host continued live coverage roya...,19
1,1,Port,Series 5: Episode 4,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,BBC ALBA,False,2020,17,music,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,bidh druthag bheag eile ’ tarraing ri chèile p...,6
2,2,"Sit Down, Stand Up with Greg James",None,Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,BBC RADIO ONE,False,2022,10,"music, entertainment",https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,tom holland greg james cold call celebrity fri...,7
3,3,Ceiliúradh na Féile Pádraig,None,John Toal and Pauline Scanlon introduce an eve...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ceiliúradh na Féile Pádraig",John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,Music,BBC TWO NORTHERN IRELAND DIGITAL,False,2017,59,music,https://ichef.bbci.co.uk/images/ic/352x198/p04...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,john toal pauline scanlon introduce evening mu...,14
4,4,Songs of Praise,Edinburgh,Claire McCollum explores Edinburgh and shares ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Songs of Praise, Edinburgh",Claire McCollum explores Edinburgh and shares ...,Claire McCollum goes on a Christian heritage t...,Claire McCollum explores Edinburgh and discove...,Music,BBC ONE,False,2022,34,"music, lifestyle",https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,claire mccollum explores edinburgh discovers r...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2468,Models: Street to Catwalk,Series 2: Episode 4,Kyra’s late arrival puts her modelling career ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Models: Street to Catwalk, S...",Kyra’s late arrival puts her modelling career ...,Kyra jeopardises her modelling career by turni...,How will Kyra cope after a train malfunction m...,Documentary,BBC THREE,Contains some strong language and some upsetti...,2021,31,documentaries,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,kyra cope train malfunction make two hour late...,4
2469,2469,Death On The I-95,None,Returning to the US to find out what happened ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Death On The I-95",Returning to the US to find out what happened ...,Following a report on the growing problem of o...,Drug overdose is now the main cause of death f...,Documentary,BBC THREE,Contains some strong language and upsetting sc...,2018,57,documentaries,https://ichef.bbci.co.uk/ima

## TF-IDF

In [3]:
# df['mixed_text'] = df['title'] + ' ' + df['description'] + ' ' + df['syno_l'] + ' ' + df['category'] + ' ' + str(df['release_date'])

In [4]:
import nltk
from nltk.corpus import stopwords
lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_text(x, stop_words):
  if pd.isna(x) == False:
    x = x.lower()
    x = x.translate(str.maketrans(' ', ' ', string.punctuation))
    x = x.strip()

    x = nltk.word_tokenize(x)
    x = [token for token in x if not token in stop_words]
    x = ' '.join([lemmatizer.lemmatize(w) for w in x])
  else:
    x = ''

  return x

In [5]:
stop_words = stopwords.words('english')
df['cleaned_syno_l'] = df['syno_l'].apply(lambda x: clean_text(x, stop_words))

In [6]:
df['cleaned_syno_l']

0       katie derham host continued live coverage roya...
1       bidh druthag bheag eile ’ tarraing ri chèile p...
2       tom holland greg james cold call celebrity fri...
3       john toal pauline scanlon introduce evening mu...
4       claire mccollum explores edinburgh discovers r...
                              ...                        
2468    kyra cope train malfunction make two hour late...
2469    drug overdose main cause death american age 50...
2470    music industry reacted surge outrage death geo...
2471    sliabh eachtaí tá cérill ag déanamh réidh mheá...
2472    2006 tim bergling regular teenager living smal...
Name: cleaned_syno_l, Length: 2473, dtype: object

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import cluster

vectorizer = TfidfVectorizer(min_df=3, max_df=0.80, norm='l2')
tf_idf_matrix = vectorizer.fit_transform(df['cleaned_syno_l'])
tf_idf = pd.DataFrame(data = tf_idf_matrix.toarray(), columns=vectorizer.get_feature_names())
tf_idf

/Users/hagenaadfock/miniforge3/envs/default/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,100,1000,10000,100000,11,12,120,12yearold,13,...,àlainn,àrdùrlar,às,èideann,òg,òga,òrain,óg,ùr,ùra
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.067534,0.0,0.056143,0.067534,0.0,0.0,0.0,0.135069,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.082349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2469,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2470,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2471,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


## K-Mean Clustering

In [61]:
clusters = 20
kmeanModel = KMeans(n_clusters=clusters, init='k-means++', max_iter=3000, random_state=0)
mod = kmeanModel.fit_transform(tf_idf)
df['k_means'] = kmeanModel.predict(tf_idf)

In [8]:
order_centroids = kmeanModel.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
dict = []
for i in range(clusters):
  print('%d' % i, sep='', end=', '),
  for ind in order_centroids[i, :20]:
    print(terms[ind], sep='', end=', ')
  print('')

0, learn, numberblocks, number, kylie, miss, routine, help, numberblock, mr, ten, 7yearolds, one, time, word, shape, friend, square, adding, count, 9yearolds, 
1, family, garden, home, life, new, mum, space, dad, help, year, make, back, house, get, time, visit, find, want, come, love, 
2, topsy, squirrel, gran, timmy, duggee, book, jojo, chloe, make, gaz, watch, tim, donna, explore, janet, play, jonny, friend, sitcom, five, 
3, find, make, help, new, go, take, friend, try, team, come, day, meanwhile, one, face, plan, life, turn, time, must, back, 
4, chef, dish, cook, masterchef, judge, gregg, one, challenge, two, celebrity, three, must, competition, four, john, contestant, week, test, kitchen, champion, 
5, car, jeremy, richard, reasonably, priced, james, clarkson, hammond, star, test, may, track, chris, motoring, quiz, week, new, round, drive, gamer, 
6, guest, week, queen, bbc, judge, show, special, joined, sewer, challenge, star, panel, strictly, look, chat, music, drag, celebrity,

/Users/hagenaadfock/miniforge3/envs/tf/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [47]:
df[df['k_means'] == 2]['title'].head(20)

2226                              North West Tonight
2233                              Reporting Scotland
2249                            Channel Islands News
2257                                        BBC News
2258                                 BBC Wales Today
2265                             East Midlands Today
2272                                      BBC London
2275                            South Today - Oxford
2282                                 BBC News at Ten
2298                                       Spotlight
2311                                Look East (West)
2314             Look North (North East and Cumbria)
2324                                South East Today
2334                                     South Today
2335    Look North (East Yorkshire and Lincolnshire)
2336                           World Business Report
2338                          Look North (Yorkshire)
2339                                        The Nine
2344                                     Point

In [49]:
df

,index,title,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic,spacied_syno_l,processed_syno_l,processed_syno_l_one_string,k_means,cleaned_syno_l
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,9pm 11 Sep 2021,5247,"['music', 'from-the-archives']",Katie Derham hosts continued live coverage fro...,"['Katie', 'Derham', 'host', 'continue', 'live'...",Katie Derham host continue live coverage Royal...,18,katie derham host continued live coverage roya...
1,1,Port,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,9 Jul 2020,1037,['music'],Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,"['Bidh', 'Druthag', 'Bheag', 'Eile', 'tarraing...",Bidh Druthag Bheag Eile tarraing ri chèile pìo...,0,bidh druthag bheag eile ’ tarraing ri chèile p...
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,False,10 Feb 2022,611,"['music', 'entertainment']",Tom Holland and Greg James cold call celebrity...,"['Tom', 'Holland', 'Greg', 'James', 'cold', 'c...",Tom Holland Greg James cold celebrity friend f...,9,tom holland greg james cold call celebrity fri...
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ceiliúradh na Féile Pádraig",John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,False,17 Mar 2017,3538,['music'],John Toal and Pauline Scanlon introduce an eve...,"['John', 'Toal', 'Pauline', 'Scanlon', 'introd...",John Toal Pauline Scanlon introduce evening mu...,0,john toal pauline scanlon introduce evening mu...
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Songs of Praise, Edinburgh",Claire McCollum explores Edinburgh and shares ...,Claire McCollum goes on a Christian heritage t...,Claire McCollum explores Edinburgh and discove...,Music,bbc_one,False,1:15pm 20 Feb 2022,2044,"['music', 'lifestyle']",Claire McCollum explores Edinburgh and discove...,"['Claire', 'McCollum', 'explore', 'Edinburgh',...",Claire McCollum explore Edinburgh discover ric...,18,claire mccollum explores edinburgh discovers r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3777,3777,Models: Street to Catwalk,Kyra’s late arrival puts her modelling career ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Models: Street to Catwalk, S...",Kyra’s late arrival puts her modelling career ...,Kyra jeopardises her modelling career by turni...,How will Kyra cope after a train malfunction m...,Documentary,bbc_three,Contains some strong language and some upsetti...,8 Aug 2021,1843,['documentaries'],How will Kyra cope after a train malfunction m...,"['Kyra', 'cope', 'train', 'malfunction', 'make...",Kyra cope train malfunction make hour crucial ...,15,kyra cope train malfunction make two hour late...
3789,3789,Death On The I-95,Returning to the US to find out what happened ...,https

In [62]:
df.to_csv('../bbc_small.csv', sep=';')

## Cosin Similarity

In [8]:
cosine = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

In [25]:
# with open('../bbc_cosine.npy', 'wb') as f:
#     np.save(f, cosine)

In [9]:
cosine

array([[1.        , 0.01713783, 0.        , ..., 0.07864291, 0.        ,
        0.02013837],
       [0.01713783, 1.        , 0.        , ..., 0.01306602, 0.13176427,
        0.0110393 ],
       [0.        , 0.        , 1.        , ..., 0.06031908, 0.00973088,
        0.        ],
       ...,
       [0.07864291, 0.01306602, 0.06031908, ..., 1.        , 0.        ,
        0.06651762],
       [0.        , 0.13176427, 0.00973088, ..., 0.        , 1.        ,
        0.        ],
       [0.02013837, 0.0110393 , 0.        , ..., 0.06651762, 0.        ,
        1.        ]])

In [10]:
df_cosine = pd.DataFrame(cosine)
df_cosine

,0,1,2,3,4,5,6,7,8,9,...,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472
0,1.000000,0.017138,0.000000,0.014938,0.027364,0.108092,0.028579,0.133374,0.029517,0.000000,...,0.000000,0.000000,0.010969,0.011114,0.000000,0.000000,0.015148,0.078643,0.000000,0.020138
1,0.017138,1.000000,0.000000,0.124017,0.018052,0.003390,0.049918,0.048575,0.018918,0.000000,...,0.000000,0.000000,0.000000,0.003028,0.000000,0.005860,0.000000,0.013066,0.131764,0.011039
2,0.000000,0.000000,1.000000,0.000000,0.000000,0.089031,0.000000,0.024758,0.015056,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011730,0.060319,0.009731,0.000000
3,0.014938,0.124017,0.000000,1.000000,0.004501,0.008006,0.009224,0.028847,0.006662,0.085284,...,0.000000,0.000000,0.000000,0.011499,0.000000,0.000000,0.000000,0.030853,0.287673,0.006592
4,0.027364,0.018052,0.000000,0.004501,1.000000,0.011410,0.018659,0.028506,0.024186,0.003457,...,0.004644,0.014201,0.013220,0.010136,0.111495,0.000000,0.025265,0.047036,0.002389,0.037674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,0.000000,0.005860,0.000000,0.000000,0.000000,0.013219,0.004256,0.022604,0.000000,0.000000,...,0.014349,0.023401,0.000000,0.007389,0.000000,1.000000,0.018303,0.008240,0.008550,0.038546
2469,0.015148,0.000000,0.011730,0.000000,0.025265,0.018788,0.002634,0.006407,0.018951,0.000000,...,0.033063,0.048845,0.000000,0.031561,0.025864,0.018303,1.000000,0.038243,0.000000,0.042264
2470,0.078643,0.013066,0.060319,0.030853,0.047036,0.039358,0.009078,0.094621,0.054468,0.000000,...,0.025582,0.020342,0.011328,0.012224,0.000000,0.008240,0.038243,1.000000,0.000000,0.066518
2471,0.000000,0.131764,0.009731,0.287673,0.002389,0.009463,0.008239,0.063299,0.012455,0.000000,...,0.000000,0.000000,0.000000,0.002986,0.000000,0.008550,0.000000,0.000000,1.000000,0.000000


In [54]:
df_cosine.loc[300].sort_values(ascending=False).index.tolist()[1:10]

[2111, 1058, 357, 191, 2014, 786, 1741, 4, 948]

In [11]:
df_cosine.to_csv('../bbc_cosine_small.csv', sep=';')

In [38]:
cosine

array([[1.        , 0.06513971, 0.0341602 , ..., 0.        , 0.        ,
        0.        ],
       [0.06513971, 1.        , 0.01425866, ..., 0.        , 0.00966852,
        0.        ],
       [0.0341602 , 0.01425866, 1.        , ..., 0.        , 0.02305491,
        0.01791652],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.00966852, 0.02305491, ..., 0.        , 1.        ,
        0.0299961 ],
       [0.        , 0.        , 0.01791652, ..., 0.        , 0.0299961 ,
        1.        ]])

In [37]:
np.sort(cosine[300, :])[::-1]

array([1.        , 1.        , 0.29444264, ..., 0.        , 0.        ,
       0.        ])